
# Glue Studio Notebook
You are now running a **Glue Studio** notebook; before you can start using your notebook you *must* start an interactive session.

## Available Magics
|          Magic              |   Type       |                                                                        Description                                                                        |
|-----------------------------|--------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| %%configure                 |  Dictionary  |  A json-formatted dictionary consisting of all configuration parameters for a session. Each parameter can be specified here or through individual magics. |
| %profile                    |  String      |  Specify a profile in your aws configuration to use as the credentials provider.                                                                          |
| %iam_role                   |  String      |  Specify an IAM role to execute your session with.                                                                                                        |
| %region                     |  String      |  Specify the AWS region in which to initialize a session                                                                                                  |
| %session_id                 |  String      |  Returns the session ID for the running session.                                                                                                          |
| %connections                |  List        |  Specify a comma separated list of connections to use in the session.                                                                                     |
| %additional_python_modules  |  List        |  Comma separated list of pip packages, s3 paths or private pip arguments.                                                                                 |
| %extra_py_files             |  List        |  Comma separated list of additional Python files from S3.                                                                                                 |
| %extra_jars                 |  List        |  Comma separated list of additional Jars to include in the cluster.                                                                                       |
| %number_of_workers          |  Integer     |  The number of workers of a defined worker_type that are allocated when a job runs. worker_type must be set too.                                          |
| %worker_type                |  String      |  Standard, G.1X, *or* G.2X. number_of_workers must be set too. Default is G.1X                                                                            |
| %glue_version               |  String      |  The version of Glue to be used by this session. Currently, the only valid options are 2.0 and 3.0 (eg: %glue_version 2.0)                                |
| %security_config            |  String      |  Define a security configuration to be used with this session.                                                                                            |
| %sql                        |  String      |  Run SQL code. All lines after the initial %%sql magic will be passed as part of the SQL code.                                                            |
| %streaming                  |  String      |  Changes the session type to Glue Streaming.                                                                                                              |
| %etl                        |  String      |   Changes the session type to Glue ETL.                                                                                                                   |
| %status                     |              |  Returns the status of the current Glue session including its duration, configuration and executing user / role.                                          |
| %stop_session               |              |  Stops the current session.                                                                                                                               |
| %list_sessions              |              |  Lists all currently running sessions by name and ID.                                                                                                     |
| %spark_conf                 |  String      |  Specify custom spark configurations for your session. E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer                       |

In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

In [ ]:
from awsglue.dynamicframe import DynamicFrame
import datetime
from pyspark.sql.functions import regexp_replace

In [ ]:
end_date = datetime.date.today()
# end_date_sat = f'{end_date}T00:00:00.000Z'
# start_date = end_date - datetime.timedelta(days = 5)
# start_date_mon = f'{start_date}T00:00:00.000Z'
# prev_date = end_date - datetime.timedelta(days = 1)

In [ ]:
# Script generated for node Amazon S3
recent_tweets_dyf = glueContext.create_dynamic_frame.from_options(
    format_options={"multiline": False},
    connection_type="s3",
    format="json",
    connection_options={
        "paths": [f"s3://dataspartan-test-bucket/twitter-analytics/raw-data/recent-tweets/results/twitter_recent_tweets_{end_date}.json"],
        "recurse": False,
    },
)

In [ ]:
recent_tweets_dyfc = recent_tweets_dyf.relationalize("root", "s3://dataspartan-test-bucket/unspecified/temp-dir/")

In [ ]:
# recent_tweets_dyfc.keys()

In [ ]:
# recent_tweets_dyfc.select('root_recent_tweets.val.entities.hashtags').toDF().show(1)

In [ ]:
recent_tweets_dyf_root_recent_tweets = recent_tweets_dyfc.select('root_recent_tweets')

In [ ]:
# Look into how to handle text
# possible solutions:
# https://docs.aws.amazon.com/athena/latest/ug/csv-serde.html
# https://docs.aws.amazon.com/athena/latest/ug/glue-best-practices.html#schema-crawlers
recent_tweets_dyf_root_recent_tweets_drop = recent_tweets_dyf_root_recent_tweets.toDF().drop('recent_tweets.val.text')

In [ ]:
recent_tweets_dyf_root_recent_tweets_drop = recent_tweets_dyf_root_recent_tweets_drop.withColumn("recent_tweets.val.created_at", regexp_replace('`recent_tweets.val.created_at`', 'T', ' '))
recent_tweets_dyf_root_recent_tweets_drop = recent_tweets_dyf_root_recent_tweets_drop.withColumn("recent_tweets.val.created_at", regexp_replace('`recent_tweets.val.created_at`', '.000Z', ''))

In [ ]:
#https://medium.com/geekculture/aws-glue-simple-job-to-write-part-csv-files-to-a-single-file-d805eddbe641
recent_tweets_dyf_root_recent_tweets_drop.repartition(1).write.mode('overwrite').csv(f's3://dataspartan-test-bucket/twitter-analytics/raw-data/recent-tweets/unnested/dump/root_recent_tweets_{end_date}',header = 'true')

In [ ]:
import boto3
client = boto3.client('s3')
BUCKET_NAME= 'dataspartan-test-bucket'
PREFIX_start ='twitter-analytics/raw-data/recent-tweets/unnested/'
PREFIX_end =f'/root_recent_tweets_{end_date}'
response = client.list_objects(
    Bucket=BUCKET_NAME, Prefix=PREFIX_start+'dump'+PREFIX_end,)
#Helps to fetch the file name of the file created in this glue job
name = response['Contents'][0]['Key']
client.copy_object(Bucket=BUCKET_NAME, 
#The csv file generated can be renamed with a name as desired !
CopySource=BUCKET_NAME+'/'+name, Key=PREFIX_start+f'root_recent_tweets/recent_tweets_single_file.csv')
# client.delete_object(Bucket=BUCKET_NAME ,Key=name)

In [ ]:
recent_tweets_dyf_val_entities_mentions = recent_tweets_dyfc.select('root_recent_tweets.val.entities.mentions')

In [ ]:
#https://medium.com/geekculture/aws-glue-simple-job-to-write-part-csv-files-to-a-single-file-d805eddbe641
recent_tweets_dyf_val_entities_mentions.toDF().repartition(1).write.mode('overwrite').csv(f's3://dataspartan-test-bucket/twitter-analytics/raw-data/recent-tweets/unnested/dump/val_entities_mentions_{end_date}',header = 'true')

In [ ]:
BUCKET_NAME= 'dataspartan-test-bucket'
PREFIX_start ='twitter-analytics/raw-data/recent-tweets/unnested/'
PREFIX_end =f'/val_entities_mentions_{end_date}'
response = client.list_objects(
    Bucket=BUCKET_NAME, Prefix=PREFIX_start+'dump'+PREFIX_end,)
#Helps to fetch the file name of the file created in this glue job
name = response['Contents'][0]['Key']
client.copy_object(Bucket=BUCKET_NAME, 
#The csv file generated can be renamed with a name as desired !
CopySource=BUCKET_NAME+'/'+name, Key=PREFIX_start+f'val_entities_mentions/entities_mentions_single_file.csv')
# client.delete_object(Bucket=BUCKET_NAME ,Key=name)

In [ ]:
recent_tweets_dyf_val_entities_hashtags = recent_tweets_dyfc.select('root_recent_tweets.val.entities.hashtags')

In [ ]:
#https://medium.com/geekculture/aws-glue-simple-job-to-write-part-csv-files-to-a-single-file-d805eddbe641
recent_tweets_dyf_val_entities_hashtags.toDF().repartition(1).write.mode('overwrite').csv(f's3://dataspartan-test-bucket/twitter-analytics/raw-data/recent-tweets/unnested/dump/val_entities_hashtags_{end_date}',header = 'true')

In [ ]:
BUCKET_NAME= 'dataspartan-test-bucket'
PREFIX_start ='twitter-analytics/raw-data/recent-tweets/unnested/'
PREFIX_end =f'/val_entities_hashtags_{end_date}'
response = client.list_objects(
    Bucket=BUCKET_NAME, Prefix=PREFIX_start+'dump'+PREFIX_end,)
#Helps to fetch the file name of the file created in this glue job
name = response['Contents'][0]['Key']
client.copy_object(Bucket=BUCKET_NAME, 
#The csv file generated can be renamed with a name as desired !
CopySource=BUCKET_NAME+'/'+name, Key=PREFIX_start+f'val_entities_hashtags/entities_hashtags_single_file.csv')
# client.delete_object(Bucket=BUCKET_NAME ,Key=name)